In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout
import numpy as np
import sympy as sp
from sympy import lambdify
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import display, Latex, Markdown, Image, Math
from scipy.integrate import solve_ivp
from scipy.signal import place_poles
import sympy as sp

In [ ]:
imag = Image("../../images/zweitank.png", width=800)
display(imag)

__System__

In [ ]:
_t, s, yf, dyf, ddyf = sp.symbols('t s yf dyf ddyf')

In [ ]:
z1 = sp.Function('z_1')
z2 = sp.Function('z_2')
uA = sp.Function('u_A')
y = sp.Function('y')
z = sp.Matrix([[z1(_t)], [z2(_t)]])

In [ ]:
dz1 = 0.00216625315586689 * uA(_t) - 0.0127646468529449 * np.sqrt(2) * sp.sqrt(z1(_t) + 0.055)
dz2 = 0.0127646468529449 * np.sqrt(2) * sp.sqrt(z1(_t) + 0.055) - 0.00908683019582126 * np.sqrt(2) * sp.sqrt(z2(_t) + 0.055)

dz = sp.Matrix([[dz1], [dz2]])
display(Latex("$\\dot{{\\mathbf{{z}}}} = {}$".format(sp.latex(dz))))

__Flacher Ausgang__

In [ ]:
display(Latex("$y(t) = z_2(t)$"))
dy = dz[1]
display(Latex("$\\dot{{y}}(t) = \\dot{{z}}_2(t) = {}$".format(sp.latex(dy))))
ddy = sp.simplify(dz[1].diff(_t).subs({z1(_t).diff(_t): dz1, z2(_t).diff(_t): dz2}))
display(Latex("$\\ddot{{y}}(t) = \\ddot{{z}}_2(t) = {}$".format(sp.latex(ddy))))

In [ ]:
_z1 = sp.solve(dy.subs({z2(_t): y(_t)}) - y(_t).diff(_t), z1(_t))[0]
display(Latex("$z_1(t) = {}$".format(sp.simplify(_z1))))

In [ ]:
display(Latex("$\\dot{{y}}(t) = \\dot{{z}}_2(t) = {}$".format(sp.latex(sp.simplify(dy.subs({
    z1(_t): _z1,
    z2(_t): y(_t)
}))))))
display(Latex("$\\ddot{{y}}(t) = \\ddot{{z}}_2(t) = {}$".format(sp.latex(sp.simplify(ddy.subs({
    z1(_t): _z1,
    z2(_t): y(_t)
}))))))

In [ ]:
_uA = sp.simplify(sp.solve(ddy.subs({z1(_t): _z1, z2(_t): y(_t)}) - y(_t).diff(_t, 2), uA(_t))[0])
display(Latex("$u_{{\\mathrm{{A}}}}(t) = {}$".format(sp.latex(_uA))))

In [ ]:
u = lambdify((yf, dyf, ddyf), _uA.subs({y(_t): yf, y(_t).diff(_t): dyf, y(_t).diff(_t,2): ddyf}))

__Definition Parammeter__

In [ ]:
tSim = np.linspace(0, 800, 801)

In [ ]:
x0 = [0.0, 0.0]
t0 = 0
yd = 0.2
dt = 250

In [ ]:
yr = lambda t: yd * (10 * (t - t0) ** 3 / dt ** 3 - 15 * (t - t0) ** 4 / dt ** 4 + 6 * (t - t0) ** 5 / dt ** 5) if t0 <= t <= t0 + dt else 0 if t < t0 else yd 
dyr = lambda t: yd * (30 * (t - t0) ** 2 / dt ** 3 - 60 * (t - t0) ** 3 / dt ** 4 + 30 * (t - t0) ** 4 / dt ** 5) if t0 < t < t0 + dt else 0
ddyr = lambda t: yd * (60 * (t - t0) / dt ** 3 - 180 * (t - t0) ** 2 / dt ** 4 + 120 * (t - t0) ** 3 / dt ** 5) if t0 < t < t0 + dt else 0

## Simulation

In [ ]:
def prozessFF(_t, x, u):
    z1 = x[0]
    z2 = x[1]
    
    uA = u(yr(_t), dyr(_t), ddyr(_t))
        
    if z1 <= 1e-6:
        hV1 = 0.0
        z1 = 0
    else:
        hV1 = 0.055

    if z2 <= 1e-6:
        hV2 = 0.0
        z2 = 0
    else:
        hV2 = 0.055
        
    dx = np.zeros(2)
    dx[0] = 0.00216625315586689 * uA - 0.0127646468529449 * np.sqrt(2) * np.sqrt(hV1 + z1)
    dx[1] = 0.0127646468529449 * np.sqrt(2) * np.sqrt(hV1 + z1) - 0.00908683019582126 * np.sqrt(2) * np.sqrt(hV2 + z2)

    return dx

In [ ]:
def prozess(_t, x, uA):
    z1 = x[0]
    z2 = x[1]
            
    if z1 <= 1e-6:
        hV1 = 0.0
        z1 = 0
    else:
        hV1 = 0.055

    if z2 <= 1e-6:
        hV2 = 0.0
        z2 = 0
    else:
        hV2 = 0.055
        
    dx = np.zeros(2)
    dx[0] = 0.00216625315586689 * uA - 0.0127646468529449 * np.sqrt(2) * np.sqrt(hV1 + z1)
    dx[1] = 0.0127646468529449 * np.sqrt(2) * np.sqrt(hV1 + z1) - 0.00908683019582126 * np.sqrt(2) * np.sqrt(hV2 + z2)

    return dx

In [ ]:
resFF = solve_ivp(prozessFF, [tSim[0], tSim[-1]], x0,
                t_eval=tSim, args=(u, ))
res = solve_ivp(prozess, [tSim[0], tSim[-1]], x0,
                t_eval=tSim, args=(3, ))

In [ ]:
plt.close()

fig, axes10 = plt.subplots(1, 1, figsize=(10,6))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False

axes10.plot(resFF.t, resFF.y[0], label=r"Höhe $z_1$")
axes10.plot(resFF.t, resFF.y[1], label=r"Höhe $z_2$")
axes10.plot(tSim, [yr(t) for t in tSim], label=r"$z_{2,\text{r}}$")
axes10.plot(resFF.t, res.y[0], label=r"Höhe $z_1$")
axes10.plot(resFF.t, res.y[1], label=r"Höhe $z_2$")

axes11 = axes10.twinx()
axes11.plot(tSim, [u(yr(t), dyr(t), ddyr(t)) for t in tSim], 'C4')
axes11.set_ylabel(r"$u_{\mathrm{A}}$ in V", color='C4')
axes11.tick_params(axis='y', colors='C4')

axes10.set_xlabel(r"$t$ in s")
axes10.set_ylabel(r"$z_{\bullet}$ in m")

axes10.grid() 

handlesAx, labelsAx = axes10.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.125, 0.90, 0.775, .15), loc=3,
           ncol=5, mode="expand", borderaxespad=0., framealpha=0.5)
plt.show()